# read partitioned data

In [2]:
from pyspark.sql import SparkSession
import os

local=True
if local:
    spark = SparkSession.builder\
        .master("local[4]")\
        .appName("RepartitionAndCoalesce")\
        .config("spark.executor.memory", "2g")\
        .getOrCreate()
else:
    spark = SparkSession.builder\
        .master("k8s://https://kubernetes.default.svc:443")\
        .appName("RepartitionAndCoalesce")\
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory","2g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

In [6]:
csv_path="../data/people.csv"

In [7]:
df=spark.read\
    .options(header=True,inferSchema=True,delimiter=',',nullValue="?")\
    .csv(path=csv_path)
df.show(5)

+---+----+---+----+---+
| id|name|sex|year|dep|
+---+----+---+----+---+
|  1|toto|  F|2002|  1|
|  2|toto|  F|2003|  1|
|  3|toto|  F|2004|  1|
|  4|titi|  M|2002|  2|
|  5|titi|  M|2003|  2|
+---+----+---+----+---+
only showing top 5 rows



In [8]:
output_path="../data/people_partiton"
df.write.partitionBy("sex","year","dep").mode("overwrite").parquet(output_path)